## CNN 국민 청원 분류하기

### 데이터 크롤링

In [5]:
for i in range(594316, 594321):
    URL = "https://www1.president.go.kr/petitions/" + str(i)
    print(URL)

https://www1.president.go.kr/petitions/594316
https://www1.president.go.kr/petitions/594317
https://www1.president.go.kr/petitions/594318
https://www1.president.go.kr/petitions/594319
https://www1.president.go.kr/petitions/594320


In [108]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver

# from selenium.webdriver.chrome.options import Options 
# options = Options() 
# options.binary_location= r'/Raid_data/home_storage/ysher_d/study/pytorch preject/'
# driver = webdriver.Chrome(r'/Raid_data/home_storage/ysher_d/study/pytorch preject/chromedriver',options = chrome_options)
# driver.implicitly_wait(3)

result = pd.DataFrame()
for i in range(584274, 595226):
    URL = "https://www1.president.go.kr/petitions/" + str(i)
    
    response = requests.get(URL)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h3', class_='petitionsView_title')
    count = soup.find('span',class_ = 'counter')
    
    for content in soup.select('div.petitionsView_write > div.View_write'):
        content
    a=[]
    for tag in soup.select('ul.petitionsView_info_list > li'):
        a.append(tag.contents[1])

    if len(a) != 0:
        df1 = pd.DataFrame({ "start" : [a[1]],
                             "end" : [a[2]],
                             "category" : [count.text],
                             "title" : [title.text],
                             "content" : [content.text.strip()[0:13000]] })

        result = pd.concat([result, df1])
        result.index = np.arange(len(result))

    if i % 60 == 0:
        print("Sleep 10seconds. Count:" + str(i)
             + ", Local Time:" + time.strftime("%Y-%m-%d", time.localtime(time.time()))
             + " " + time.strftime("%X", time.localtime(time.time()))
             + ", Data Length:"+str(len(result))
             )
        time.sleep(1)

Sleep 10seconds. Count:584280, Local Time:2021-11-08 15:04:13, Data Length:7
Sleep 10seconds. Count:584340, Local Time:2021-11-08 15:04:22, Data Length:59
Sleep 10seconds. Count:584400, Local Time:2021-11-08 15:04:26, Data Length:59
Sleep 10seconds. Count:584460, Local Time:2021-11-08 15:04:29, Data Length:59
Sleep 10seconds. Count:584520, Local Time:2021-11-08 15:04:33, Data Length:59
Sleep 10seconds. Count:584580, Local Time:2021-11-08 15:04:37, Data Length:59
Sleep 10seconds. Count:584640, Local Time:2021-11-08 15:04:40, Data Length:59
Sleep 10seconds. Count:584700, Local Time:2021-11-08 15:04:44, Data Length:59
Sleep 10seconds. Count:584760, Local Time:2021-11-08 15:04:47, Data Length:59
Sleep 10seconds. Count:584820, Local Time:2021-11-08 15:04:51, Data Length:59


KeyboardInterrupt: 

In [112]:
result = pd.read_csv("../data/2020_petition.csv", index_col=[0])
print(result.shape)

df = result
df.head()

(10887, 5)


,start,end,category,title,content
0,2020-01-02,2020-02-01,267,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...,본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 ...
1,2020-01-02,2020-02-01,271,주식시장 활성화 및 소액(개미)투자자 보호,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...
2,2020-01-02,2020-02-01,198,교정기관의 민낮,억울한 일로 국민청원을 신청합니다.\n\n 저는 **구치소 **교도관입니다.\n\n...
3,2020-01-02,2020-02-01,170,미세먼지 저감 대책,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다.\n우리 일상에서 가장...
4,2020-01-02,2020-02-01,"2,127",악질세입자 방지를 위한 세입자보호법을 재정해주세요.,저는 우선 아이셋의 부모입니다.\n식구가 많은편이고 아이들이 성장함에 따라 집이 좁...


### 데이터 전처리

In [113]:
import re

def remove_white_space(text):
    text = re.sub(r"[\t\r\n\f\v]"," ",str(text))
    return text

def remove_special_char(text):
    text = re.sub("[^ ㄱ-ㅣ가-힣 0-9]+"," ",str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

### 토크나이징 및 변수 생성

In [116]:
from konlpy.tag import Okt

okt = Okt()

df['title_token'] = df.title.apply(okt.morphs) # 형태소로 토크나이징
df['content_token'] = df.content.apply(okt.nouns) # 명사로 토크나이징

### 파생변수 생성

In [118]:
df.head()

,start,end,category,title,content,title_token,content_token,token_final
0,2020-01-02,2020-02-01,267,서울지방병무청 탈의실에 설치된 에 대한 진상규명을 요구한다 또한 인권위의 미온적...,본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 ...,"[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...","[본인, 경, 서울, 지방, 병무청, 제, 검사, 탈의실, 수, 것, 발견, 탈의실...","[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,..."
1,2020-01-02,2020-02-01,271,주식시장 활성화 및 소액 개미 투자자 보호,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...,"[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호]","[우리, 나라, 코스피, 총, 미국, 애플, 설, 돌, 정도, 한국, 주식시장, 평...","[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호, 우리, 나라, 코스피, 총..."
2,2020-01-02,2020-02-01,198,교정기관의 민낮,억울한 일로 국민청원을 신청합니다 저는 구치소 교도관입니다 2015 ...,"[교정, 기관, 의, 민낮]","[일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이주, 전, 구치소, 독, 거...","[교정, 기관, 의, 민낮, 일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이..."
3,2020-01-02,2020-02-01,170,미세먼지 저감 대책,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다 우리 일상에서 가장 ...,"[미세먼지, 저, 감, 대책]","[미세먼지, 심각, 성은, 이제, 적극, 대안, 요구, 우리, 일상, 가장, 적극,...","[미세먼지, 저, 감, 대책, 미세먼지, 심각, 성은, 이제, 적극, 대안, 요구,..."
4,2020-01-02,2020-02-01,"2,127",악질세입자 방지를 위한 세입자보호법을 재정해주세요,저는 우선 아이셋의 부모입니다 식구가 많은편이고 아이들이 성장함에 따라 집이 좁아...,"[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...","[저, 우선, 아이, 셋, 부모, 식구, 편이, 아이, 집, 나름, 꿈, 가지, 좀...","[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ..."


In [124]:
df['token_final'] = df.title_token + df.content_token
df['count'] = df["category"].replace({",":""}, regex=True).apply(lambda x : int(x))
print(df.dtypes)

start            object
end              object
category         object
title            object
content          object
title_token      object
content_token    object
token_final      object
count             int64
dtype: object


In [125]:
df["label"] = df["count"].apply(lambda x: "Yes" if x>=1000 else "No")

In [126]:
df_drop = df[["token_final","label"]]

### 단어임베딩

In [141]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop["token_final"], sg=1, vector_size=100, window=2, min_count=1, workers=4) # sg=1 : Skip-Gram, sg=0 : CBOW
print(embedding_model)

model_result = embedding_model.wv.most_similar("음주운전")
print(model_result)

Word2Vec(vocab=44424, vector_size=100, alpha=0.025)
[('음주', 0.8832599520683289), ('뺑소니', 0.8329351544380188), ('무면허', 0.8204187154769897), ('강력범죄', 0.774462103843689), ('살인자', 0.7694199085235596), ('살인죄', 0.7628235220909119), ('전과자', 0.7483433485031128), ('운전자', 0.7382883429527283), ('승자', 0.7381764650344849), ('윤창', 0.728901207447052)]


### 임베딩 모델 저장 및 로드

In [145]:
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('../model/petitions_tokens_w2v')

loaded_model = KeyedVectors.load_word2vec_format("../model/petitions_tokens_w2v")

model_result = loaded_model.most_similar("음주운전")

### 데이터셋 분할 및 저장

In [146]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('../data/petition_train.csv', index=False, encoding="utf-8-sig")
val.to_csv('../data/petition_validation.csv', index=False, encoding="utf-8-sig")

### Field 클래스 정의

In [165]:
import torchtext
from torchtext.legacy.data import Field # 추상화, iterater, 토크나이징, 단어장 생성기능 제공

def tokenizer(text):
    text = re.sub("[\[\]\']","",str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize = tokenizer)
LABEL = Field(sequential = False)

### 데이터 불러오기

In [193]:
from torchtext.legacy.data import TabularDataset

train, validation = TabularDataset.splits(path = "../data/", train = "petition_train.csv",validation = "petition_validation.csv",
                                          format="csv",fields=[("text",TEXT), ("label",LABEL)], skip_header = True)

print("Train:", train[0].text, train[0].label)
print("validation:", validation[0].text, validation[0].label)

Train: ['이제', '직원', '들', '에게', '사직', '을', '권고', '하려', '합니다', '이제', '직원', '권고사직', '저', '아내', '행업', '주로', '해외', '용역', '중심', '사업', '추진', '결과', '창업', '지방', '소재', '여행사', '로서', '나라장터', '통', '학교', '공공기관', '꽤', '수주', '이', '부문', '인정', '코로나', '사태', '여파', '지난', '이후', '모든', '수주', '취소', '현재', '신규', '수주', '직원', '해고', '수', '사정', '정부', '특별', '고용', '안정', '원금', '지난', '개', '월간', '직원', '모두', '유급', '휴직', '저', '아내', '둘이서', '사무실', '출근', '사업', '모색', '해', '그동안', '사무실', '임차료', '사무실', '이전', '개월', '인건비', '회사', '부담', '액', '사무실', '임차료', '각종', '사무실', '회사', '유지', '공과금', '세금', '등', '소상', '공인', '긴급', '대출', '금도', '모두', '소진', '상태', '더', '수', '상황', '정부', '고용', '안정', '원금', '개월', '더', '국외여행', '업', '재개', '기약', '상황', '고용', '유지', '것', '이상', '직원', '급여', '부담', '보험료', '무기', '퇴직금', '충당금', '감안', '때', '감당', '직원', '개월', '연장', '기대', '줄', '알', '직원', '권고사직', '후', '실업', '급여', '수령', '모색', '해', '보라', '것', '그동안', '정부', '고용', '률', '유', '위', '막대', '예산', '투여', '고용', '유지', '원금', '제도', '행업', '근로자', '도움', '조치', '영주', '입장', '사업', '재개', '언제', '상황',

### 단어장 및 DataLoader 정의

In [204]:
import torch
from torchtext.vocab import Vectors # 임베딩 벡터를 만들기위한 클래스
from torchtext.legacy.data import BucketIterator # 배치사이즈 만큼 데이터를 로드하는 iterator

vectors = Vectors(name='../model/petitions_tokens_w2v')

TEXT.build_vocab(train, vectors=vectors, min_freq=1, max_size=None) # 단어장 만들기, 최소 등장횟수, 전체 사이즈 제한
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(datasets=(train,validation), batch_size=32, device = device, sort=False)

print("임베딩 벡터의 개수와 차원: {}".format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원: torch.Size([40641, 100])


### TextCNN 모델 설계

In [224]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TextCNN(nn.Module):
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)
        self.convs = nn.ModuleList([nn.Conv1d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()
        self.Dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)
        
    def forward(self, x):
        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)
        con_x = [self.relu(conv(emb_x)) for conv in self.convs]
        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]
        
        fc_x = torch.cat(pool_x, dim=1)
        fc_x = fc_x.squeeze(-1)
        fc_x = self.Dropout(fc_x)
        logit = self.fc(fc_x)
        return logit

### 모델 학습 함수 정의

In [225]:
def train(model, device, train_itr, optimizer):
    
    model.train()
    corrects, train_loss = 0.0, 0
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        optimizer.zero_grad()
        logit = model(text)
        loss = F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)
    
    return train_loss, accuracy

### 모델 평가를 위한 함수 정의

In [226]:
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0 ,0
    
    for batch in itr:
        
        text, target = batch.text, batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)
        
        test_loss += loss.item()
        result = torch.max(logit, 1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
        
    test_loss /= len(itr.dataset)
    accuracy = 100*corrects/len(itr.dataset)
    
    return test_loss, accuracy
        

### 모델 학습 및 성능 확인

In [227]:
model = TextCNN(vocab, 100, 10, [3,4,5], 2).to(device)
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1,3+1):
    tr_loss, tr_acc = train(model, device, train_iter, optimizer)
    print("Train Epoch: {} \t Loss: {} \t Accuracy: {}%".format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    print(f"Valid Epoch: {epoch}, \t Loss: {val_loss} \t Accuracy: {val_acc}%")
    
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    print("------------------------------------------------------------------------------")

TextCNN(
  (embed): Embedding(40641, 100)
  (convs): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(3, 100), stride=(1,))
    (1): Conv1d(1, 10, kernel_size=(4, 100), stride=(1,))
    (2): Conv1d(1, 10, kernel_size=(5, 100), stride=(1,))
  )
  (relu): ReLU()
  (Dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch: 1 	 Loss: 0.020739015652856924 	 Accuracy: 61.125144958496094%
Valid Epoch: 1, 	 Loss: 0.01965135616559636 	 Accuracy: 65.50299072265625%
model saves at 65.50299072265625 accuracy
------------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.01934735302467981 	 Accuracy: 66.90011596679688%
Valid Epoch: 2, 	 Loss: 0.019477819186260133 	 Accuracy: 67.34037780761719%
model saves at 67.34037780761719 accuracy
------------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.017884630909196 	 Accuracy: 71.0792236328125%
Valid Epoch: 3, 	 Loss: